### Часть 1.1
### Реализация парсера

In [17]:
import time
from tqdm import tqdm
from bs4 import BeautifulSoup
from selenium import webdriver
from dataclasses import dataclass
import seaborn as sns
import pandas as pd
import random
import requests
import nltk
nltk.download("stopwords")

sns.set(style="darkgrid")
%matplotlib inline

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kukof\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
DEPTH = 20
SLEEP = random.randint(1,5)
BASE_URL = 'https://www.gazeta.ru/'
TOPICS = ['politics' , 'social', 'army', 'culture', 'science', 'family', 'tech', 'style', 'business', 'civilization']

In [19]:
@dataclass
class Article:
    id: str = None
    url: str = None
    source: str = None
    title: str = None
    content: str = None
    datetime: str = None

In [20]:
# set webdriver params
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('headless')
chrome_options.add_argument('no-sandbox')
chrome_options.add_argument('disable-dev-shm-usage')
driver = webdriver.Chrome(options=chrome_options)

# driver = webdriver.Chrome('./chromedriver')

In [21]:
def get_pages():

    """Load and scroll pages"""

    items, topics_order = [], []

    for topic in tqdm(TOPICS):
        try:
            old_size = len(items)
            URL = BASE_URL + topic
            driver.get(URL)
            time.sleep(SLEEP)

     # We can adjust this number to get more posts
            last_height = driver.execute_script("return document.body.scrollHeight")

            NUM_SCROLLS = 5000

            for i in range(NUM_SCROLLS):

                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(SLEEP)
                driver.execute_script("document.getElementsByClassName('b_showmorebtn')[0].click()")
                time.sleep(SLEEP)

                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height

            html = driver.page_source
            soup = BeautifulSoup(html, "html.parser")
            items += soup.find_all('a', {'class' : 'b_ear'})
            new_size = len(items)
            if new_size > old_size:
                topics_order.extend([topic] * (new_size - old_size))

        except:
            pass

    return items, topics_order



In [22]:
def parse_page(page):
    """Extract from page desired fields"""

    # Create article data class object
    article = Article()

    # article id
    article.id = page['href']

    # article url
    article.url = 'https://www.gazeta.ru' + article.id

    # load page
    driver.get(article.url)
    time.sleep(random.randint(1,5))
    html = driver.page_source

    #soup = BeautifulSoup(html, "html.parser")
    response = requests.get(article.url)
    soup = BeautifulSoup(response.content, 'html.parser', from_encoding='utf-8')

    # article source
    article.source = article.url[12: article.url.find('.')]

    # process article title
    article.title = soup.find('h1', {'class': 'headline'}).text


    # article content
    article.content = soup.find('div', {'class': 'b_article-text'}).text

    # article datetime
    str_ = soup.find('time', {'class': 'time'})['datetime']
    article.datetime = str_[:str_.find('T')]

    return article

In [23]:

# get pages and topics
pages, topics_order = get_pages()
# with open('pages.txt', 'w') as f:
#     for page in pages:
#         f.write("%s\n" % page)

100%|██████████| 10/10 [1:41:12<00:00, 607.23s/it]


In [24]:
len(pages)

3989

In [25]:
# parse each page and get desired attributes
   # few pages are differ significantly from others
# using try/except we can ignore them
data, topics_order_fixed = [], []
for num, page in enumerate(tqdm(pages)):
    try:
        res = parse_page(page)
        data.append(res)
        topics_order_fixed.append(topics_order[num])
    except:
        pass

driver.close()

100%|██████████| 3989/3989 [24:42:57<00:00, 22.31s/it]    


In [33]:
df_gazeta = pd.DataFrame(data=data)
df_gazeta['topic'] = topics_order_fixed
df_gazeta

,id,url,source,title,content,datetime,topic
0,/politics/news/2023/11/01/21617029.shtml,https://www.gazeta.ru/politics/news/2023/11/01...,,Верховный суд впервые за 100 лет обновит понят...,Пленум Верховного суда (ВС) готовит постановле...,2023-11-01,politics
1,/politics/news/2023/11/01/21616981.shtml,https://www.gazeta.ru/politics/news/2023/11/01...,,Белый дом: Байден наложит вето на законопроект...,"В случае, если палата представителей конгресса...",2023-11-01,politics
2,/politics/news/2023/11/01/21617005.shtml,https://www.gazeta.ru/politics/news/2023/11/01...,,"В США заявили, что Вашингтон не даст закончить...",Экс-советник Пентагона полковник в отставке Ду...,2023-11-01,politics
3,/politics/news/2023/11/01/21616933.shtml,https://www.gazeta.ru/politics/news/2023/11/01...,,Венесуэла осудила удар Израиля по лагерю бежен...,Венесуэла решительно осуждает удар Армии оборо...,2023-11-01,politics
4,/politics/news/2023/11/01/21616951.shtml,https://www.gazeta.ru/politics/news/2023/11/01...,,Колумбия и Чили отзывают послов из Израиля,Президент Колумбии Густаво Петро заявил в соцс...,2023-11-01,politics
...,...,...,...,...,...,...,...
3496,/comments/column/articles/17803639.shtml,https://www.gazeta.ru/comments/column/articles...,,Почему говорить «падик» — не кринж,"\n«Ну я же тебе не Хатико, чтоб вечно ждать те...",2023-10-31,science
3497,/comments/column/bovt/17796697.shtml,https://www.gazeta.ru/comments/column/bovt/177...,,Выпишет ли Америка Украине чек на $60 млрд?,\nАдминистрация Джо Байдена запросила дополнит...,2023-10-30,science
3498,/comments/column/articles/17791651.shtml,https://www.gazeta.ru/comments/column/articles...,,«Параллельная реальность». Почему школьники не...,"\nЕсли люди — это новая нефть, то молодежь — э...",2023-10-28,science
3499,/comments/column/articles/17787793.shtml,https://www.gazeta.ru/comments/column/articles...,,Ориентация на Китай,"\nРоссиян всегда интересовали инструменты, защ...",2023-10-27,science


In [77]:
df_gazeta.duplicated().sum(), len(df_gazeta.drop_duplicates())

(0, 3501)

In [79]:
df_gazeta.source = df_gazeta.url.str[12:18]
df_gazeta

,id,url,source,title,content,datetime,topic
0,/politics/news/2023/11/01/21617029.shtml,https://www.gazeta.ru/politics/news/2023/11/01...,gazeta,Верховный суд впервые за 100 лет обновит понят...,Пленум Верховного суда (ВС) готовит постановле...,2023-11-01,politics
1,/politics/news/2023/11/01/21616981.shtml,https://www.gazeta.ru/politics/news/2023/11/01...,gazeta,Белый дом: Байден наложит вето на законопроект...,"В случае, если палата представителей конгресса...",2023-11-01,politics
2,/politics/news/2023/11/01/21617005.shtml,https://www.gazeta.ru/politics/news/2023/11/01...,gazeta,"В США заявили, что Вашингтон не даст закончить...",Экс-советник Пентагона полковник в отставке Ду...,2023-11-01,politics
3,/politics/news/2023/11/01/21616933.shtml,https://www.gazeta.ru/politics/news/2023/11/01...,gazeta,Венесуэла осудила удар Израиля по лагерю бежен...,Венесуэла решительно осуждает удар Армии оборо...,2023-11-01,politics
4,/politics/news/2023/11/01/21616951.shtml,https://www.gazeta.ru/politics/news/2023/11/01...,gazeta,Колумбия и Чили отзывают послов из Израиля,Президент Колумбии Густаво Петро заявил в соцс...,2023-11-01,politics
...,...,...,...,...,...,...,...
3496,/comments/column/articles/17803639.shtml,https://www.gazeta.ru/comments/column/articles...,gazeta,Почему говорить «падик» — не кринж,"\n«Ну я же тебе не Хатико, чтоб вечно ждать те...",2023-10-31,science
3497,/comments/column/bovt/17796697.shtml,https://www.gazeta.ru/comments/column/bovt/177...,gazeta,Выпишет ли Америка Украине чек на $60 млрд?,\nАдминистрация Джо Байдена запросила дополнит...,2023-10-30,science
3498,/comments/column/articles/17791651.shtml,https://www.gazeta.ru/comments/column/articles...,gazeta,«Параллельная реальность». Почему школьники не...,"\nЕсли люди — это новая нефть, то молодежь — э...",2023-10-28,science
3499,/comments/column/articles/17787793.shtml,https://www.gazeta.ru/comments/column/articles...,gazeta,Ориентация на Китай,"\nРоссиян всегда интересовали инструменты, защ...",2023-10-27,science


In [80]:
df_gazeta.to_pickle('df_gazeta.p', compression='gzip')